In [7]:
import fitz # PyMuPDF
from llama_index.core import Document
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.mistralai import MistralAIEmbedding

/Users/simon/Documents/venvs/versare/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Function to read PDF content
def parse_pdf(file_path):
    pdf_document = fitz.open(file_path)
    text = ""
    
    # Extract text from each page
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()

    pdf_document.close()
    return text


In [9]:
# Path to the PDF file
pdf_path = "your_path"  # Replace with your PDF file

In [10]:
# Parse the PDF and get the text content
pdf_text = parse_pdf(pdf_path)

In [11]:
# Create a LlamaIndex Document
document = Document(text=pdf_text)

In [4]:
api_key = ""
model_name = "mistral-embed"
mistral_embed_model = MistralAIEmbedding(model_name=model_name, api_key=api_key)

In [26]:
# Parameters that can be tuned later to test different models and different chunk sizes
model_embedder_name = "Alibaba-NLP/gte-large-en-v1.5"
# Settings.embed_model = mistral_embed_model
Settings.embed_model = HuggingFaceEmbedding(model_name=model_embedder_name, trust_remote_code=True)
# Settings.llm = None
# Settings.chunk_size = 1024
# Settings.chunk_overlap = 20

Settings.text_splitter = SentenceSplitter(chunk_size=128, chunk_overlap=20)

In [ ]:
index = VectorStoreIndex.from_documents(documents=[document])

# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [21]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.2)],
)

In [22]:
# query documents - testing
query = """quel est l'article qui traite les dispositions dérogatoires ?"""
response = query_engine.query(query)

In [23]:
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"
    context += "-" * 50 + "\n\n"

print(context)

Context:
Article 2 -  Possibilité de dérogation 
 
Il peut être dérogé à certaines dispositions du présent Cahier des clauses administratives générales 
moyennant que ces dispositions soient expressément mentionnées, à peine de nullité de la 
dérogation, dans le Cahier des clauses administratives particulières (CCAP). 
 
 
 
Section 2 – Intervenants dans les marchés de travaux 
 
 
Article 3 - Maître d'ouvrage 
 
3.1. Le maître d'ouvrage est la personne morale publique ou l'autorité contractante pour le compte de 
laquelle les travaux sont exécutés, à qui revient l'ouvrage, et qui, parmi tous les intervenants, a seul le 
pouvoir de décision. 
 
3.2.

--------------------------------------------------

sauf dérogation autorisée et mention expresse au 
Cahier des clauses administratives particulières (CCAP) ; 
9. les frais de bornage, de morcellement et de piquetage ; 
10. l’établissement, le fonctionnement et l’entretien des clôtures, des dispositifs de sécurité et des 
installations d’